In [63]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e3/sample_submission.csv
/kaggle/input/playground-series-s4e3/train.csv
/kaggle/input/playground-series-s4e3/test.csv


In [64]:
df = pd.read_csv('/kaggle/input/playground-series-s4e3/train.csv')

In [65]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

In [66]:
from sklearn.model_selection import train_test_split,GridSearchCV


In [67]:
df.head()

,id,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,0,584,590,909972,909977,16,8,5,2274,113,...,-0.5000,-0.0104,0.1417,0,0,0,1,0,0,0
1,1,808,816,728350,728372,433,20,54,44478,70,...,0.7419,-0.2997,0.9491,0,0,0,0,0,0,1
2,2,39,192,2212076,2212144,11388,705,420,1311391,29,...,-0.0105,-0.0944,1.0000,0,0,1,0,0,0,0
3,3,781,789,3353146,3353173,210,16,29,3202,114,...,0.6667,-0.0402,0.4025,0,0,1,0,0,0,0
4,4,1540,1560,618457,618502,521,72,67,48231,82,...,0.9158,-0.2455,0.9998,0,0,0,0,0,0,1


In [68]:
X = df.drop(columns = ['id','Pastry', 'Z_Scratch', 'K_Scatch', 'Stains',
       'Dirtiness', 'Bumps', 'Other_Faults'])
y = df[['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains',
       'Dirtiness', 'Bumps', 'Other_Faults']]


In [69]:
rf = RandomForestClassifier(random_state=1)
mo = MultiOutputClassifier(rf,n_jobs = -1)

In [70]:
param_grid = {
    "estimator__max_depth":[80,90,100],
    "estimator__n_estimators" :[10,13,15]
}

grid = GridSearchCV(mo,param_grid = param_grid,cv = 3,n_jobs = -1,scoring = 'roc_auc')


In [71]:
grid.fit(X,y)

GridSearchCV(cv=3,
             estimator=MultiOutputClassifier(estimator=RandomForestClassifier(random_state=1),
                                             n_jobs=-1),
             n_jobs=-1,
             param_grid={'estimator__max_depth': [80, 90, 100],
                         'estimator__n_estimators': [10, 13, 15]},
             scoring='roc_auc')

In [72]:
grid.best_score_

0.8357744279893621

In [73]:
model = grid.best_estimator_

In [82]:
df_test = pd.read_csv('/kaggle/input/playground-series-s4e3/test.csv')

In [83]:
df_test.shape

(12814, 28)

In [84]:
X_test = df_test.drop(columns = ['id'])
X_test.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Outside_X_Index,Edges_X_Index,Edges_Y_Index,Outside_Global_Index,LogOfAreas,Log_X_Index,Log_Y_Index,Orientation_Index,Luminosity_Index,SigmoidOfAreas
0,1015,1033,3826564,3826588,659,23,46,62357,67,127,...,0.0095,0.5652,1.0000,1.0,2.8410,1.1139,1.6628,0.6727,-0.2261,0.9172
1,1257,1271,419960,419973,370,26,28,39293,92,132,...,0.0047,0.2414,1.0000,1.0,2.5682,0.9031,1.4472,0.9063,-0.1453,0.9104
2,1358,1372,117715,117724,289,36,32,29386,101,134,...,0.0155,0.6000,0.7500,0.0,2.4609,1.3222,1.3222,-0.5238,-0.0435,0.6514
3,158,168,232415,232440,80,10,11,8586,107,140,...,0.0037,0.8000,1.0000,1.0,1.9031,0.6990,1.0414,0.1818,-0.0738,0.2051
4,559,592,544375,544389,140,19,15,15524,103,134,...,0.0158,0.8421,0.5333,0.0,2.1461,1.3222,1.1461,-0.5714,-0.0894,0.4170


In [86]:
pred_y = model.predict(X_test)

In [88]:
pred_y

AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [99]:
pred_y = pd.DataFrame(pred_y)
pred_y['id'] = df_test['id']
# final = pd.concat([
#     pd.Series(df_test['id']),
#     pd.Series(pred_y)
# ],axis = 1)

In [100]:
pred_y

,0,1,2,3,4,5,6,id
0,0,0,0,0,0,0,0,19219
1,0,0,0,0,0,0,1,19220
2,0,0,0,0,0,0,0,19221
3,0,0,0,0,0,1,0,19222
4,0,0,0,0,0,1,0,19223
...,...,...,...,...,...,...,...,...
12809,0,0,0,0,0,0,0,32028
12810,0,0,0,0,0,0,1,32029
12811,0,0,1,0,0,0,0,32030
12812,1,0,0,0,0,0,1,32031


In [113]:
pred_y = pred_y.set_index('id')

In [114]:
pred_y

,0,1,2,3,4,5,6
id,,,,,,,
19219,0,0,0,0,0,0,0
19220,0,0,0,0,0,0,1
19221,0,0,0,0,0,0,0
19222,0,0,0,0,0,1,0
19223,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...
32028,0,0,0,0,0,0,0
32029,0,0,0,0,0,0,1
32030,0,0,1,0,0,0,0


In [122]:
pred_y.to_csv('output1.csv')